In [1]:
import pandas as pd
import numpy as np
import gensim
from helpers import lemmatize_stemming, preprocess, compute_coherence_values

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
import matplotlib.pyplot as plt
%matplotlib inline


filenames = !ls data/unzipped
filenames = [ 'data/unzipped/'+filename for filename in filenames ]

import random
indeces = random.sample(range(len(filenames)), 50)
# indeces = [68]

/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/ipykernel/pylab/config.py:70: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):
/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/matplotlib_inline/config.py:66: DeprecationWarning: InlineBackend._figure_formats_changed is deprecated in traitlets 4.1: use @observe and @unobserve instead.
  def _figure_formats_changed(self, name, old, new):


In [2]:
from helpers import load_tweets

documents = load_tweets(np.array(filenames)[indeces])

/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
processed_docs = [ preprocess(doc) for doc in documents ]
dictionary = gensim.corpora.Dictionary(processed_docs)
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
tfidf = gensim.models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]


/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=14, id2word=dictionary, passes=10, workers=4, alpha=0.01, eta=.91)

for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Topic: 0 
Words: 0.006*"plant" + 0.006*"meat" + 0.005*"prison" + 0.005*"halfway" + 0.005*"test" + 0.005*"jersey" + 0.004*"posit" + 0.004*"tyson" + 0.003*"worker" + 0.003*"domest"
Topic: 1 
Words: 0.008*"protest" + 0.007*"michigan" + 0.005*"arm" + 0.004*"vaccin" + 0.003*"capitol" + 0.003*"cure" + 0.003*"follow" + 0.003*"scam" + 0.003*"covid" + 0.003*"polic"
Topic: 2 
Words: 0.003*"kill" + 0.002*"totallockdown" + 0.002*"cyjramhtx" + 0.002*"comparison" + 0.002*"fair" + 0.002*"disinfect" + 0.002*"critic" + 0.002*"greet" + 0.001*"permit" + 0.001*"tonyposnanski"
Topic: 3 
Words: 0.006*"year" + 0.005*"time" + 0.005*"lik

In [5]:
pyLDAvis.enable_notebook()
vis = gensimvis.prepare(lda_model, corpus_tfidf, dictionary)
vis

/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7     -0.072087  0.021500       1        1  18.296654
4     -0.093795 -0.021401       2        1  14.934978
11    -0.059355  0.038500       3        1  14.557495
9      0.004802 -0.086993       4        1   8.820708
6     -0.021668 -0.002998       5        1   8.308369
3     -0.030092 -0.013909       6        1   8.217182
8     -0.013306  0.005298       7        1   6.718535
12     0.009321  0.035312       8        1   6.103800
13     0.040274 -0.035318       9        1   3.749641
1      0.038946  0.006806      10        1   2.912799
10     0.039884  0.007533      11        1   2.281726
0      0.047215  0.018703      12        1   2.262575
5      0.053449  0.014879      13        1   1.629321
2      0.056412  0.012087      14        1   1.206216, topic_info=           Term        Freq       Total Category  logprob  loglift
42         case  319.000000  319.000000  Default  30.0000  30.0000
41         test  364.000000  364.000000  Default  29.0000  29.0000
44        death  320.000000  320.000000  Default  28.0000  28.0000
6         trump  274.000000  274.000000  Default  27.0000  27.0000
145       china  182.000000  182.000000  Default  26.0000  26.0000
...         ...         ...         ...      ...      ...      ...
3086   malaysia    1.239916   13.713560  Topic14  -6.9310   2.0143
497      brazil    1.460018   25.343980  Topic14  -6.7676   1.5636
441       robot    1.211847   17.720156  Topic14  -6.9539   1.7351
1331  misinform    1.184453   18.990569  Topic14  -6.9767   1.6430
234         say    1.205648  263.973240  Topic14  -6.9590  -0.9711

[766 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
2453      1  0.139273  aarogya
2453      2  0.139273  aarogya
2453      3  0.139273  aarogya
2453      4  0.139273  aarogya
2453      5  0.139273  aarogya
...     ...       ...      ...
1497      5  0.036990     zone
1497      6  0.036990     zone
1497      8  0.036990     zone
1497     11  0.369901     zone
1497     12  0.036990     zone

[5803 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 5, 12, 10, 7, 4, 9, 13, 14, 2, 11, 1, 6, 3])

In [6]:
len(documents)

/home/adrien/twitter-analysis/nlpenv/lib/python3.8/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


37626